In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
def get_rev_compilemt(seq):
    if len(seq) != 30:
        print(len(seq))
        raise ValueError("gRNA must be 30 bases long")
    rev_grna = seq[::-1]
    rev_comp_grna = ""
    for base in rev_grna:
        if base.upper() == "A":
            rev_comp_grna += "T"
        elif base.upper() == "T":
            rev_comp_grna += "A"
        elif base.upper() == "C":
            rev_comp_grna += "G"
        elif base.upper() == "G":
            rev_comp_grna += "C"
        else:
            raise ValueError("Invalid base in gRNA")
    return rev_comp_grna

In [3]:
after_crisproff_path = 'After CRISPRoff/CRISPRparams.tsv'
after_crisproff_full_guides = 'After CRISPRoff/crispron.csv'
before_crisproff_path = 'Combined Files/co_and_kim_30.csv'

before_crisproff_df = pd.read_csv(before_crisproff_path)
after_crisproff_df = pd.read_csv(after_crisproff_path, sep='\t')
after_crisproff_full_guides_df = pd.read_csv(after_crisproff_full_guides)

# merge the two dataframes based on the ID or guideID column
merged_df = after_crisproff_df.merge(after_crisproff_full_guides_df, left_on='guideID', right_on='ID')
#merged_df.to_csv('After CRISPRoff/crispron_pretrain_with_crisproff.csv', index=False)

rows = []
for row in merged_df.iterrows():
    sequence = row[1]['30mer']
    crispr_off_score = row[1]['CRISPRoff_score']

    if sequence in before_crisproff_df['Sequence'].values:
        total_indel_eff_rescaled = before_crisproff_df[before_crisproff_df['Sequence'] == sequence]['total_indel_eff_rescaled'].values[0]
        rows.append([sequence, crispr_off_score, total_indel_eff_rescaled])
    else:
        rev_comp_sequence = get_rev_compilemt(sequence)
        if rev_comp_sequence in before_crisproff_df['Sequence'].values:
            total_indel_eff_rescaled = before_crisproff_df[before_crisproff_df['Sequence'] == rev_comp_sequence]['total_indel_eff_rescaled'].values[0]
            rows.append([sequence, crispr_off_score, total_indel_eff_rescaled])

df = pd.DataFrame(rows, columns=['Sequence', 'CRISPRoff_score', 'total_indel_eff_rescaled'])
df.to_csv('After CRISPRoff/co_data.csv', index=False)
